In [1]:
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from html_table_parser import parser_functions as parse
from time import sleep
import pandas as pd

### Get a list of all counties in WI

In [2]:
# Get the list of all counties in WI
county_url = "https://dnr.wi.gov/lakes/clmn/"
county_html = requests.get(county_url)

In [3]:
# .text returns the request content in Unicode
county_html.text[:500]

'\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n\r\n<head><meta http-equiv="content-type" content="text/html;charset=utf-8" /><link href="http://dnr.wi.gov/favicon.ico" type="image/x-icon" rel="icon" /><link href="http://dnr.wi.gov/favicon.ico" type="image/x-icon" rel="shortcut icon" /><title>\r\n\tCitizen Lake Monitoring Network\r\n</title><!-- BEGIN global_head.inc ( /includes/global_head'

In [4]:
county_soup = bs(county_html.text, 'html.parser')

In [5]:
county_soup.find_all(name='li', attrs={'class':'multiColList'})

[<li class="multiColList"><a href="Stations.aspx?location=1">
                 Adams County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=2">
                 Ashland County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=3">
                 Barron County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=4">
                 Bayfield County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=5">
                 Brown County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=6">
                 Buffalo County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=7">
                 Burnett County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=8">
                 Calumet County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=9"

In [6]:
# Extract list of all counties, correctly formatted 
counties = []
for entry in county_soup.find_all(name='li', attrs={'class':'multiColList'}):
    counties.append(entry.text.replace('\r\n', '').replace(' ', '').replace('County',' County'))
print(len(counties))

72


### Get all reports containing CHLA readings

Get all station numbers

In [25]:
# find station number in from table column station ID 
# All counties: https://dnr.wi.gov/lakes/clmn/Stations.aspx?location=0 
# By county: https://dnr.wi.gov/lakes/clmn/Stations.aspx?location=1 

driver = webdriver.Firefox()
station_url = 'https://dnr.wi.gov/lakes/clmn/Stations.aspx?location=0'
driver.get(station_url)

stationIDs = []

def stationID_extract(pageNumber):
    for i in range(pageNumber):
        station = driver.page_source
        station_soup = bs(station, "html.parser")
        station_table = station_soup.find('table',  { "class" : "greysuitsyou" })
        station_twod_array = parse.make2d(station_table)
        for n in range(2,len(station_twod_array)-1):
            stationIDs.append(station_twod_array[n][1])
        #sleep(1)
        # go to next page
        driver.find_element_by_id("ctl00_ctl00_LeftPageContent_gvStationTable_ctl01_LinkButton3").click()
    

In [26]:
stationID_extract(40)
driver.close()

In [27]:
print(stationIDs)

['504001', '443052', '643047', '703060', '183082', '184002', '443053', '553068', '10040059', '643220', '163120', '643042', '433364', '493229', '643444', '10031332', '643401', '493104', '213142', '433035', '433120', '013159', '10028945', '10028946', '013037', '10031430', '10031431', '683297', '023124', '093053', '10021451', '10039551', '10049055', '10022318', '663045', '493056', '493058', '493057', '583164', '043122', '673119', '10012421', '10020976', '443373', '043174', '433354', '033182', '443324', '073067', '073122', '663112', '453282', '494002', '383209', '643121', '353077', '563058', '663050', '693104', '10049162', '663128', '443187', '033139', '433372', '213186', '693041', '433247', '493122', '443121', '693106', '593067', '143311', '033137', '143122', '033134', '033131', '143034', '033130', '033133', '10048426', '10049214', '033132', '10048427', '083044', '233117', '10039772', '643193', '433300', '643050', '663167', '373416', '073063', '493144', '493106', '443056', '673206', '6732

In [216]:
# Report url example: stationNo, year1=1950&year2=2017 (regardless of most recent data)
# http://dnrx.wisconsin.gov/swims/public/reporting.do?type=58&action=post&stationNo=013159&year1=1950&year2=2017&format=csv
# http://dnrx.wisconsin.gov/swims/public/reporting.do?type=58&action=post&stationNo=10021087&year1=1950&year2=2017&format=csv
# http://dnrx.wisconsin.gov/swims/public/reporting.do?type=58&action=post&stationNo=013178&year1=1950&year2=2017&format=csv
# http://dnrx.wisconsin.gov/swims/public/reporting.do?type=58&action=post&stationNo=273120&year1=1950&year2=2017&format=csv



In [28]:
# Get the reports

report_url = 'http://dnrx.wisconsin.gov/swims/public/reporting.do?type=58&action=post&stationNo=' + \
             str(stationIDs[1]) + '&year1=1950&year2=2017&format=csv'
print(report_url)

import urllib.request

original_report_name = './data/wi-lakes/report_'+str(stationIDs[1])+'.csv'
urllib.request.urlretrieve(report_url, original_report_name)  


http://dnrx.wisconsin.gov/swims/public/reporting.do?type=58&action=post&stationNo=443052&year1=1950&year2=2017&format=csv


('./data/wi-lakes/report_443052.csv',
 <http.client.HTTPMessage at 0x26681115ba8>)

Clean downloaded reports

In [29]:
# Clean report file

f = open(original_report_name,'r')
lines = f.readlines()
f.close()
goodlines = []
start = 0
for i in range(len(lines)):
    if "Chlorophyll" in lines[i]:
        start = i
    if start > 0:
        if lines[i].strip().replace(",","") == "":
            goodlines = lines[start:i]
            break


In [39]:
wbic = lines[4].split(',')[2]
print(wbic)

967400


In [33]:
cleanlines=[]
for i in range(len(goodlines)):
    cleanlines.append(goodlines[i].replace('1-Beautiful, could not be nicer', '1-beautiful'))
print("\n".join(cleanlines))
    

Group Seq No, Start Date, Secchi (Feet),Secchi Hit Bottom?,Secchi (Meters),Chlorophyll(ug/l),Total Phosphorus(ug/l),Secchi TSI,Total Phosphorus TSI,Chlorophyll TSI,Lake Level,Staff Gauge,Appearance,Color,Perception

1,08/16/1979,,,,,,,,,,,,,

7000068,08/29/2001,,,,5.8,,,,48,,,,,

8188599,08/29/2001,3,NO,0.9,,,61,,,,,,,

1,08/14/2007,5.1,NO,1.55,3.54,29,54,54,44,,,,,

83293435,08/17/2013,4.75,NO,1.4,,,55,,,NORMAL,3,CLEAR,BROWN,2-Very minor aesthetic problems

83293435,09/03/2013,4,NO,1.2,,,57,,,NORMAL,,CLEAR,BROWN,2-Very minor aesthetic problems

83293435,07/01/2014,4.25,NO,1.3,,,56,,,HIGH,,CLEAR,BROWN,1-beautiful

83293435,07/10/2014,4.25,NO,1.3,,,56,,,HIGH,,CLEAR,BROWN,2-Very minor aesthetic problems

83293435,08/02/2014,4.25,NO,1.3,,,56,,,NORMAL,,CLEAR,BROWN,2-Very minor aesthetic problems

83293435,08/08/2014,5,NO,1.5,,,54,,,NORMAL,,CLEAR,BROWN,2-Very minor aesthetic problems

83293435,08/31/2014,4, ,1.2,,,57,,,NORMAL,,CLEAR,GREEN,2-Very minor aesthetic problems

83293435,09/09/2014

In [70]:
clean_report_name = './data/wi-lakes/report_'+str(stationIDs[1])+'clean.csv'   

f = open(clean_report_name,'w')      
f.writelines(cleanlines)
f.close()


In [80]:
df = pd.read_csv(clean_report_name,sep=",")

In [82]:
# clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '_').str.replace(')', '')
df['wbic'] = wbic
df.head()

,group_seq_no,start_date,secchi__feet,secchi_hit_bottom?,secchi__meters,chlorophyll_ug/l,total_phosphorus_ug/l,secchi_tsi,total_phosphorus_tsi,chlorophyll_tsi,lake_level,staff_gauge,appearance,color,perception,wbic
0,1,08/16/1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,967400
1,7000068,08/29/2001,NaN,NaN,NaN,5.80,NaN,NaN,NaN,48.0,NaN,NaN,NaN,NaN,NaN,967400
2,8188599,08/29/2001,3.00,NO,0.90,NaN,NaN,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,967400
3,1,08/14/2007,5.10,NO,1.55,3.54,29.0,54.0,54.0,44.0,NaN,NaN,NaN,NaN,NaN,967400
4,83293435,08/17/2013,4.75,NO,1.40,NaN,NaN,55.0,NaN,NaN,NORMAL,3.0,CLEAR,BROWN,2-Very minor aesthetic problems,967400


In [83]:
df.drop(['perception'], axis=1)

,group_seq_no,start_date,secchi__feet,secchi_hit_bottom?,secchi__meters,chlorophyll_ug/l,total_phosphorus_ug/l,secchi_tsi,total_phosphorus_tsi,chlorophyll_tsi,lake_level,staff_gauge,appearance,color,wbic
0,1,08/16/1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,967400
1,7000068,08/29/2001,NaN,NaN,NaN,5.80,NaN,NaN,NaN,48.0,NaN,NaN,NaN,NaN,967400
2,8188599,08/29/2001,3.00,NO,0.90,NaN,NaN,61.0,NaN,NaN,NaN,NaN,NaN,NaN,967400
3,1,08/14/2007,5.10,NO,1.55,3.54,29.0,54.0,54.0,44.0,NaN,NaN,NaN,NaN,967400
4,83293435,08/17/2013,4.75,NO,1.40,NaN,NaN,55.0,NaN,NaN,NORMAL,3.0,CLEAR,BROWN,967400
5,83293435,09/03/2013,4.00,NO,1.20,NaN,NaN,57.0,NaN,NaN,NORMAL,NaN,CLEAR,BROWN,967400
6,83293435,07/01/2014,4.25,NO,1.30,NaN,NaN,56.0,NaN,NaN,HIGH,NaN,CLEAR,BROWN,967400
7,83293435,07/10/2014,4.25,NO,1.30,NaN,NaN,56.0,NaN,NaN,HIGH,NaN,CLEAR,BROWN,967400
8,83293435,08/02/2014,4.25,NO,1.30,NaN,NaN,56.0,NaN,NaN,NORMAL,NaN,CLEAR,BROWN,967400
9,83293435,08/08/2014,5.00,NO,1.50,NaN,NaN,54.0,NaN,NaN,NORMAL,NaN,CLEAR,BROWN,967400


### extras

In [ ]:
from html_table_parser import parser_functions as parse

station_url = 'https://dnr.wi.gov/lakes/clmn/Stations.aspx?location=0'
station_html = requests.get(station_url)

station_soup = bs(station_html.text, "html.parser")
station_table = station_soup.find('table',  { "class" : "greysuitsyou" })
station_twod_array = parse.make2d(station_table)

# print 2D array
print(len(station_twod_array))

stations=[]
for i in range(2,len(station_twod_array)-1):
    stations.append(station_twod_array[i][1])

print(stations)

In [ ]:
# get station ID
stationIDs = []
def access_chla_reports():
    for i in range(0,len(counties)):
        station_url = 'https://dnr.wi.gov/lakes/clmn/Stations.aspx?location='+str(i+1)
        #print(station_url)
        station_html = requests.get(station_url)
        station_soup = bs(station_html.text, "html.parser")
        station_table = station_soup.find('table',  { "class" : "greysuitsyou" })
        if station_table == None:
            print('location '+str(i+1) + ' ' + counties[i] + ' has no lakes.')
        else:
            station_twod_array = parse.make2d(station_table)
            for n in range(2,len(station_twod_array)-1):
                stationIDs.append(station_twod_array[n][1])
            #print('location '+str(i+1) + ' ' + counties[i] + ' has ' + str(len(station_twod_array)) + ' stations.')